# Lab 1: Classification, some n-gram math and tf-idf

Due date: Jan 29th, 2024, 11:59 PM

Agenda
------
+ Detecting the end of a sentence
    - Rule-based classifier
+ Detecting the sentiment of a sentence
    - Rule-based classifier (counting words)
    - Measuring Accuracy, Precision, Recall (evaluating a classifier)
+ N-gram Math 
+ tf-idf


Classification can take several forms. Here are some vocabulary terms to get you started:

- __classifier__: a model that takes data (text, in NLP) as input and outputs a category
- __binary classification__: a model that takes input and outputs *one of two* categories (e.g. "positive" or "negative")
- __multinomial classification__: a model that takes input and outputs *one of many* categories (e.g. "positive", "neutral" or "negative" or a language model that chooses one token from the entire vocabulary)


- __rule-based classifier__: a classifier that functions based on rules that humans come up with (e.g. "the end of a sentence is when there is a "." ")
- __statistical classifier__: a classifier that functions based on counts (statistics) that it has gathered or based on running an algorithm to automatically train parameters on a given data set. 
    
In this lab, you'll be building rule-based classifiers and evaluating them. We'll learn about our first statistical classifier next lecture.


# Task 0: Who is in your group?

__Ashutosh Rane__

__Sanjana J D__

__Nikita Mandal__

Task 1: Tokenizer
-------


In [140]:

def tokenize(s: str) -> list:
    """
    Tokenize a string based on whitespace
    Parameters:
        s - string piece of text
    returns a list of strings from the text. 
    Each item is an individual linguistic unit. 
    """
    # TODO: fill in this function
    # you may find using the str.split() functions
    # helpful
    # you may make whatever decisions make sense to you/your 
    # group about how to tokenize.
    # don't spend longer than ~5 minutes implementing this function.
    return s.split()

test_string = "fill me in with whatever you want."
tokenized = tokenize(test_string)
print(tokenized)

['fill', 'me', 'in', 'with', 'whatever', 'you', 'want.']


1. What decisions does your tokenizer make about what should/should not be a token? __It separates the tokens based on white space only.__

In [141]:
# read in the text of moby dick (ensure the txt file is in the same directory as this notebook)
# if you do not already have - link to download text http://www.gutenberg.org/files/2701/2701-0.txt 
# right click and 'save as' into the directory this notebook is located as 'moby_dick.txt'
moby = open('mobydick.txt', "r", encoding='utf-8')

print(moby.readline()) # first line is blank
print(moby.readline()) # second line just to see if its correct
moby.close()

# now read in the full contents
moby = open('mobydick.txt', "r", encoding='utf-8')
contents = moby.read()
moby.close()

print(len(contents)) # how long is this string?



The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman

1238243


In [142]:
# call your tokenize function on the contents of moby dick
tokens = tokenize(contents)
print("Number of tokens:",len(tokens))
print("Vocabulary length:",len(set(tokens)))

Number of tokens: 215829
Vocabulary length: 33584


2. How many tokens are in *Moby Dick* when you use your `tokenize` function on its contents? __215829__
3. How big is the __vocabulary__ of *Moby Dick* when you use your `tokenize` function on its contents? __33584__

# Task 2: Detecting the end of a sentence


A classifier is, in essence, a function that takes some data $x$ and assigns some label $y$ to it. For a binary classifier, we can model this a function that takes a data point $x$ and returns either `True` or `False`.

Later in this class we'll learn about how to build classifiers that automatically learn how to do this, but we'll start where NLP started—writing some rule-based classifiers.

In [143]:
def classify_sentence_end(text: str, target_index: int) -> bool: 
    """
    Classify whether or not a *location* is the end of a sentence within
    a given text
    Parameters:
        text - string piece of text
        target_index - int candidate location
    returns true if the target index is the end of a sentence. 
    False otherwise. 
    """
    # TODO: write a simple, rule-based classifier that
    # decides whether or not a specific location is the 
    # end of a sentence
    if text[target_index] == ("." or "?" or "!"):
        return True
    else:
        return False
    # pass 

# look at the code in the cell below to see example usage

In [144]:
# example text
# feel free to go through different examples

# This is the given example text
"""Stocks were up as advancing issues outpaced declining issues 
          on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong, 
          while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among 
          individual stocks, the two top percentage gainers in the S.&P. 500 
          were Incyte Corporation and Gilead Sciences Inc."""

example = "Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.5 to 1. Large- and small-cap stocks were both strong, while the S.&P. 500 index gained 0.46% to finish at 2,457.59. Among individual stocks, the two top percentage gainers in the S.&P. 500 were Incyte Corporation and Gilead Sciences Inc."

# this code will go through and
# build up a string based on the sentence
# decisions that your classifier comes up with
# it will put "****" between the sentences
# you do not need to modify any code here
so_far = ""
sentence_count = 0
for index in range(len(example)):
    # see how the classify_sentence_end function is called!
    result = classify_sentence_end(example, index)
    so_far += example[index]
    if result:
        sentence_count += 1
        print(so_far)
        print("****")
        so_far = ""
        
print(so_far)
print("total number of sentences:", sentence_count)

Stocks were up as advancing issues outpaced declining issues on the NYSE by 1.
****
5 to 1.
****
 Large- and small-cap stocks were both strong, while the S.
****
&P.
****
 500 index gained 0.
****
46% to finish at 2,457.
****
59.
****
 Among individual stocks, the two top percentage gainers in the S.
****
&P.
****
 500 were Incyte Corporation and Gilead Sciences Inc.
****

total number of sentences: 10


1. How many sentences are detected using your end of sentence classifier? **10**
2. Where did your end of sentence classifier make a mistake? **It counted the decimal points or the periods that are used as abbreviations also as sentence endings.**

Task 3: install `nltk`
-----

If you finish the first two tasks, work on making sure that you have `nltk` downloaded and accessible to your jupyter notebooks. While you will not be allowed to use `nltk` for most of your homework, we will use it frequently in class to demonstrate tools. 

[`nltk`](https://www.nltk.org/) (natural language toolkit) is a python package that comes with many useful implementations of NLP tools and datasets.

From the command line, using pip: `pip3 install nltk` or `pip install nltk`

[installing nltk](https://www.nltk.org/install.html)

In [145]:
import nltk
# for the tokenizers that we're going to use
# won't cause an error if you've already downloaded it
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ashra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [146]:
example = "N.K. Jemison is a science fiction author. Prof. Felix isn't."
words = nltk.word_tokenize(example)
print(words)

['N.K', '.', 'Jemison', 'is', 'a', 'science', 'fiction', 'author', '.', 'Prof.', 'Felix', 'is', "n't", '.']


In [147]:
moby_nltk_tokens = nltk.word_tokenize(contents)
# feel free to add/edit code
# print(moby_nltk_tokens)
print(len(set(moby_nltk_tokens)))

22257


1. How does the size of the vocabulary for Moby Dick compare when you use `nltk`'s tokenizer vs. the one that you made? 

__The size of the vocabulary decresed for nltk tokenizer compared to the tokenizer that I made. The size of vocabulary for NLTK tokenizer is 22257 whereas for the tokenizer that I made, the length was 33584__

Task 4: Determining Sentiment
----

In [148]:
# we'll use nltk to access the reviews that we want to classify eventually
import nltk
import nltk.corpus as corpus

In [149]:
def load_word_list(filename):
    """
    Loads a lexicon from a plain text file in the format of one word per line.
    Parameters:
    filename (str): path to file

    Returns:
    list: list of words
    """
    with open(filename, 'r', encoding="utf-8") as f:
        # skip the header content
        for line in f:
            if line.strip() == "":
                break
        # read the rest of the lines into a list
        return [line.strip() for line in f]
    # otherwise return an empty list
    return []

In [150]:
# load in the positive and negative word lists here
# TODO: the paths to your negative/positive word files here
neg_lex = load_word_list("negative_words.txt")
pos_lex = load_word_list("positive_words.txt")

# TODO: How many words are in each list?
print("Postive words length-",len(pos_lex))

print("Negative words length-",len(neg_lex))


# TODO: Use python's list slicing to look at the first 10 elements in each list
print(neg_lex[:10])
print(pos_lex[:10])

Postive words length- 2006
Negative words length- 4783
['2-faced', '2-faces', 'abnormal', 'abolish', 'abominable', 'abominably', 'abominate', 'abomination', 'abort', 'aborted']
['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation']


In [151]:
# TODO: which words are in both the positive and the negative lists?
common_words = [common for common in neg_lex if common in pos_lex]
print(len(common_words))
print(common_words)

3
['envious', 'enviously', 'enviousness']


Now, we'll create our rule-based classifier! We have access to the word lists that you loaded and anything else you know about the world (reflect on how you as a human being can tell if a review is positive/negative). Your classifier need not be perfect, but it should be reasonable (don't just say everything is positive!).

In [152]:
def print_verbose(message, verbose):
     if verbose:
          print(message)
def rule_based_classify(tokens, pos_lexicon, neg_lexicon, verbose = False):
    """
    This function classifies a given tokenized text as positive or negative
    based on the provided lexicons.
    Parameters:
    tokens (list) - list of strings tokenized words in the text to classify
    pos_lexicon (list) - list of strings words in the positive word lexicon
    neg_lexicon (list) - list of strings words in the negative word lexicon
    verbose (boolean) - flag indicating whether or not to print verbose (debugging) output. 
            Default value False.
    Returns:
    string "pos" if the list of tokens is positive overall, "neg" if they are negative overall.
    """
    
    # TODO: implement this function! This is our classifier.
    positive_count = 0
    negative_count = 0
    for token in tokens:
        if token in pos_lex:
             positive_count += 1
        if token in neg_lex:
             negative_count += 1
    

    print_verbose(f"Postive count {positive_count}", verbose)
    print_verbose(f"Negative Count {negative_count}", verbose)
    if positive_count >= negative_count:
         return "pos"
    else:
         return "neg"
    


In [153]:
# now, we'll test out your classifier!
# Here are two example movie reviews.
nltk.download('movie_reviews')
movies = corpus.movie_reviews

# load in a single negative review
negative_toks = movies.words('neg/cv001_19502.txt')
# uncomment the text below to see the contents of the review
neg_text = " ".join(negative_toks)
print(neg_text)

# load in a single positive review
positive_toks = movies.words('pos/cv992_11962.txt')
pos_text = " ".join(positive_toks)
print(pos_text)



# TODO:
# call your rule_based_classify on these example reviews.

result = rule_based_classify(negative_toks, neg_lexicon = neg_lex, pos_lexicon= pos_lex, verbose = True)
print("Classification for Negative Tok:",result)

result = rule_based_classify(positive_toks, neg_lexicon = neg_lex, pos_lexicon = pos_lex, verbose = False  )
print("Classification for Positive Tok:",result)


# Does our classification function label them correctly? Why or why not?
# take a look at the contents of the reviews

the happy bastard ' s quick movie review damn that y2k bug . it ' s got a head start in this movie starring jamie lee curtis and another baldwin brother ( william this time ) in a story regarding a crew of a tugboat that comes across a deserted russian tech ship that has a strangeness to it when they kick the power back on . little do they know the power within . . . going for the gore and bringing on a few action sequences here and there , virus still feels very empty , like a movie going for all flash and no substance . we don ' t know why the crew was really out in the middle of nowhere , we don ' t know the origin of what took over the ship ( just that a big pink flashy thing hit the mir ) , and , of course , we don ' t know why donald sutherland is stumbling around drunkenly throughout . here , it ' s just " hey , let ' s chase these people around with some robots " . the acting is below average , even from the likes of curtis . you ' re more likely to get a kick out of her work i

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ashra\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


1. What labels does your classifier assign these two reviews? __My Classifier assigns both the test cases as Positive__
2. Are these correct? __The First test case is a negative review as the end of the review says "it ' s pretty much a sunken ship of a movie.". So my classifier is giving inaccurate classification results.__

Task 5: How good is your sentiment classifier?
-----

Given the movies dataset from `nltk`, how many of the reviews does your classifier classify correctly?

We'll look at three different metrics: __accuracy__, __precision__, and __recall__.

__accuracy__: what you think of when you think of correctness.
$$ \frac{\texttt{number correct}}{\texttt{total number}}$$

Precision and recall require differentiated between the ways in which the classifier can be correct or incorrect. 

- __true positive__: an example whose gold label is positive and that the classifier labels as positive
- __true negative__: an example whose gold label is negative and that the classifier labels as negative
- __false positive__: an example whose gold label is negative and that the classifier labels as positive
- __false negative__: an example whose gold label is positive and that the classifier labels as negative

In [154]:
import random
# you can use numpy's random functionality if you'd like to
import numpy as np

In [155]:
# To see the available file ids, this is one way that we can access them.
# This will give you a list of neg/positive file ids.
print(len(movies.fileids('neg')))
# choose 100 random items without replacement from a list
print(random.sample(movies.fileids('neg'), 100))
print(len(movies.fileids('pos')))

1000
['neg/cv538_28485.txt', 'neg/cv436_20564.txt', 'neg/cv358_11557.txt', 'neg/cv918_27080.txt', 'neg/cv681_9744.txt', 'neg/cv055_8926.txt', 'neg/cv513_7236.txt', 'neg/cv281_24711.txt', 'neg/cv563_18610.txt', 'neg/cv611_2253.txt', 'neg/cv037_19798.txt', 'neg/cv454_21961.txt', 'neg/cv994_13229.txt', 'neg/cv857_17527.txt', 'neg/cv569_26750.txt', 'neg/cv589_12853.txt', 'neg/cv693_19147.txt', 'neg/cv132_5423.txt', 'neg/cv379_23167.txt', 'neg/cv911_21695.txt', 'neg/cv896_17819.txt', 'neg/cv586_8048.txt', 'neg/cv966_28671.txt', 'neg/cv609_25038.txt', 'neg/cv866_29447.txt', 'neg/cv003_12683.txt', 'neg/cv019_16117.txt', 'neg/cv884_15230.txt', 'neg/cv430_18662.txt', 'neg/cv306_10859.txt', 'neg/cv269_23018.txt', 'neg/cv752_25330.txt', 'neg/cv241_24602.txt', 'neg/cv541_28683.txt', 'neg/cv247_14668.txt', 'neg/cv068_14810.txt', 'neg/cv328_10908.txt', 'neg/cv931_18783.txt', 'neg/cv913_29127.txt', 'neg/cv066_11668.txt', 'neg/cv973_10171.txt', 'neg/cv671_5164.txt', 'neg/cv498_9288.txt', 'neg/cv746_10

In [156]:
# TODO:
# Write code that uses your classifier to classify 100 randomly chosen
# negative reviews and 100 randomly chosen positive reviews
# count the number of true positives, true negatives, false positives, and false negatives

# to get the tokens associated with a certain file id,
# tokens = movies.words(file_id)

# takes a long time to run if you loop over all fileids as opposed to just
# 100 randomly chosen ones
# make sure you don't classify the same review twice!
# (it takes us about 10 seconds to classify 200 reviews on a 2020 macbook air)
# movies.fileids('neg')
random_neg_list = random.sample(movies.fileids('neg'), 100)
random_pos_list = random.sample(movies.fileids('pos'), 100)

result_pos = [result for result in random_pos_list]

result_actual_pos = [rule_based_classify(movies.words(review), neg_lexicon=neg_lex, pos_lexicon=pos_lex, verbose=False) for review in random_pos_list]
tp = result_actual_pos.count("pos")
fn = result_actual_pos.count("neg")
print("True Postive:",tp)
print("False Negative:", fn)
result_actual_pos = [rule_based_classify(movies.words(review), neg_lexicon=neg_lex, pos_lexicon=pos_lex, verbose=False) for review in random_neg_list]

tn = result_actual_pos.count("neg")
fp = result_actual_pos.count("pos")
print("True Negative:",tn)
print("False Positive:",fp)





# print(random_list)





    
# TODO: print out the number of true positives, false positives,
# false negatives, and true negatives


True Postive: 69
False Negative: 31
True Negative: 81
False Positive: 19


Here are the equations for accuracy, precision, and recall in terms of what we've just been counting. $tp$ means true positive, $fp$ means false positive, $fn$ means false negative, and $tn$ means true negative.

$$ accuracy = \frac{tp + tn}{tp + fp + fn + tn}$$

$$ precision = \frac{tp}{tp + fp}$$

$$ recall = \frac{tp}{tp + fn}$$

You can think of precision as "how many of my positive guesses were correct?" and recall as "how many of the positive examples did I find?" 😄

In [157]:
# TODO: calculate and print accuracy
accuracy = (tp+tn)/(tp+fp+fn+tn)
print("Accuracy is:",accuracy)

Accuracy is: 0.75


In [158]:
# TODO: calculate and print precision

precision = tp/(tp+fp)
print(f"Precision is: {precision:.2f}")

Precision is: 0.78


In [159]:
# TODO: calculate and print recall
recall = tp/(tp+fn)
print(f"Recall is {recall :.2f}")


Recall is 0.69


Task 6: n-gram math
----



In [160]:
import time
from collections import Counter

def count_list(ls: list) -> dict:
    counts = {}
    for item in ls:
        # we're not going to be clever about counting here,
        # no conditionals, no sets, nothing
        counts[item] = ls.count(item)
    return counts

# see the difference between the following two items
example = [random.randint(0, 100) for i in range(2000)]
start = time.time()
count_list(example)
end = time.time()
print("That took:", end - start, "seconds!")

# this takes a very similar amount of time to count_dict from HW 1
start = time.time()
Counter(example)
end = time.time()
print("That took:", end - start, "seconds!")

That took: 0.03499913215637207 seconds!
That took: 0.0 seconds!


In [161]:
# TODO: put your create_ngrams (or make_ngrams) function here!
def make_ngrams(tokens: list, n: int) -> list:
    """Creates n-grams for the given token sequence.
    Args:
    tokens (list): a list of tokens as strings
    n (int): the length of n-grams to create

    Returns:
    list: list of tuples of strings, each tuple being one of the individual n-grams
    """
    # TODO: implement this function!
    return [tuple(tokens[i:i+n]) for i in range(len(tokens) - n + 1)]
    pass

In [162]:
# TODO: calculate the bigram score of the following sequence of tokens
# for this example, we'll use a "vanilla" scoring technique
# no Laplace smoothing, no unknown tokens
training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]

# TODO: call your create_ngrams function to get your bigrams

train_bigrams = make_ngrams(training_data, 2)
vocab = count_list(train_bigrams)
to_score = ["<s>", "I", "love", "cats", "</s>"]
start = time.time()

# BEGIN SCORING SECTION
# start probability at one so that we can multiply the probability of
# each subsequent next token with it
test_bigram = make_ngrams(to_score,2)
total_prob = 1
for i in range(0, len(test_bigram)):
    # TODO: YOUR SCORE CALCULATION CODE HERE
    total_prob *= vocab[test_bigram[i]]/training_data.count(to_score[i])

# END SCORING SECTION
end = time.time()

# print your final probability
print("Final probability:", total_prob)
print("That took", end - start, "seconds!")

Final probability: 0.3333333333333333
That took 0.0015087127685546875 seconds!


In [163]:
# Finally, pretend that we had a lot more data
training_data = ["<s>", "I", "love", "dogs", "</s>", "<s>", "I", "love", "cats", "</s>", "<s>", "I", "love", "dinosaurs", "</s>"]
# this is the amount of training data in the berp set(https://www1.icsi.berkeley.edu/Speech/berp.html)
training_data = training_data * 3778

# TODO: call your create_ngrams function here
train_bigrams2 = make_ngrams(training_data, 2)

print("Number of training tokens:", len(training_data))
start = time.time()
total_prob = 1
# and what if we had 5000 sentences to score?
for example_num in range(5000):
    # TODO: COPY AND PASTE YOUR SCORING CODE HERE (between "BEGIN SCORING SECTION" and "END SCORING SECTION")
    test_bigram = make_ngrams(to_score,2)
    total_prob = 1
    for i in range(0, len(test_bigram)):
    # TODO: YOUR SCORE CALCULATION CODE HERE
        total_prob *= vocab[test_bigram[i]]/training_data.count(to_score[i])
    # (remove any print statements that you have)
    # (make sure it is appropriately indented)

    
    

end = time.time()
print("That took", end - start, "seconds!")

Number of training tokens: 56670
That took 7.9273340702056885 seconds!


What's the moral of the story? If you perform your counts at the same time you score, you'll be doing the same work over and over again which will result in a significantly slower model!

Make sure that you're gathering the counts that you need in `train` and only performing scoring calculations (as opposed to also counting things) in `score`.

This is particularly important when using larger data sets! (berp is not that big)

Task 7: Calculating tf-idf
----

Recall our discussion of tf-idf vector representations of documents in a corpus. Here, we will learn how to compute the tf-idf values for each token in a document. To calculate tf-idf, we'll need to first construct a term-document matrix from our data.

In [164]:
import numpy as np

In [165]:
def tf(term: str, document: list) -> float:
    """
    Calculate term frequency
    Parameters:
    term - string
    document - list of strings (tokenized document)
    Return:
    float term frequency
    """
    return np.log10(document.count(term) + 1)

def idf(N: int, df_t: int) -> float:
    """
    Calculate inverse document frequency given the 
    number of documents and the number of documents the term appears in.
    Paramenters:
    N - int (number of documents)
    df_t - int (number of documents the term appears in)
    Return:
    float inverse document frequency
    """
    return np.log10(N / df_t)


def term_in_documents(term: str, documents: list) -> int:
    """
    Calculate the number of documents in a list of documents that a target
    term appears in.
    Parameters:
    term - str
    documents - list of list of str (list of tokenized documents)
    Return:
    int number of documents the term appears in
    """
    #TODO: IMPLEMENT ME
    count = 0
    for doc in documents:
        if term in doc:
            count+=1

    return count


# load in the data
def load_tokens(filename):
    f = open(filename, 'r', encoding="utf-8")
    contents = f.read().lower()
    f.close()
    # if you don't have nltk installed, use another tokenization
    # strategy here like str.split()
    return nltk.word_tokenize(contents)

mobydick = load_tokens('mobydick.txt') 
shakes = load_tokens('shakespeare.txt')
pandp = load_tokens('prideandprejudice.txt')
books = [mobydick, shakes, pandp]

In [166]:
# we won't create the entire term-document matrix, we'll just do it for a few key terms that we
# care about for the sake of time
# TODO: pick 3 - 5 words
words = ['jupiter','beauty','rose','parsley','good']

In [167]:
# create the term-document matrix
matrix = []

# iterate through your chosen words
for word in words:
    # count how many times it occurs in each book
    # this is just for debugging for you
    counts = [str(b.count(word)) for b in books]
    print(word, ":\t", "\t".join(counts))
    
    
    # calculate the term frequency for each book
    # for this word
    # tf_words should be the same length as counts
    tf_words = [tf(word,book) for book in books]

    
    # calculate idf for this term
    # this will be a single scalar
    N = len(books)
    df_t = term_in_documents(word, books)
    idf_t = idf(N,df_t)
    
    # multiply tf with idf for each book/each
    # term frequency you calculated
    tfidf_words = [tf_word*idf_t for tf_word in tf_words ]
    
    # add the tfidf numbers to your matrix
    matrix.append(tfidf_words)
    
    # uncomment to see visually the different components (helpful for debugging)
    #print(word, " tf:\t", "\t".join([str(x) for x in tf_words]))
    #print(word, "idf:\t", idf_t)
    #print(word, " tf-idf:\t", "\t".join([str(x) for x in tfidf_words]))
    
# if you'd like to, uncomment the following code to make the matrix into a numpy array
matrix = np.array(matrix)
print(matrix)

jupiter :	 4	34	0
beauty :	 8	228	26
rose :	 23	64	7
parsley :	 1	1	0
good :	 195	2793	180
[[0.12308251 0.27189689 0.        ]
 [0.         0.         0.        ]
 [0.         0.         0.        ]
 [0.05300875 0.05300875 0.        ]
 [0.         0.         0.        ]]



1. What should the dimensions of your matrix be? 

__The words list has 5 elements and books has 3, therefore our matrix should be a 5x3 matrix__

2. What happens if you attempt to calculate tfidf of a term that exists in none of your books? 

__If the term does not exist in the books, the code throws Dividing by zero error. Since df_t = 0 and idf = np.log10(N / df_t) will be dividing by 0, hence the error.__

2. What happens if you attempt to calculate tfidf of a term that exists in all of your books? 

__If the term exists in all the books then N = df_t and log(N/df_t) = 0. tfidf will be 0.__

In [168]:
# check the dimensions of your matrix
# number of rows should match number of words
# number of cols should match number of documents (books)
print(matrix.shape)

(5, 3)


In [169]:
def cosine_sim(v1: list, v2: list) -> float:
    """
    Calculate the cosine similarity between two vectors of the same length.
    Parameters:
    v1 - list (of numbers)
    v2 - list (of numbers)
    Return:
    float cosine similarity
    """
    # you may find the numpy functions np.dot() and np.linalg.norm() useful
    # TODO: implement
    dot_product = np.dot(v1, v2)
    norm_v1 = np.linalg.norm(v1)
    norm_v2 = np.linalg.norm(v2)
    
    if norm_v1 == 0 or norm_v2 == 0:
        return 0  # Avoid division by zero
    
    similarity = dot_product / (norm_v1 * norm_v2)
    return similarity

# calculate the similarity between two word vectors
# we'll just do word vectors because unless matrix is a numpy array
# it is (more) difficult to get column vectors

word_vectors = {
    'tiger': np.array([0.2, 0.8, -0.3]),
    'lion': np.array([0.9, 0.4, -0.6]),
    'cat': np.array([-0.4, 0.3, 0.7])
}
similarity_tiger_lion = cosine_sim(word_vectors['tiger'], word_vectors['lion'])
print("Cosine Similarity between 'tiger' and 'lion':", similarity_tiger_lion)

Cosine Similarity between 'tiger' and 'lion': 0.6719511934699012


Optional Section (no extra credit)
----

In [170]:
# if you finish, which book is closest to moby dick?
# you'll need a *column* vector here (instead of a row vector)


In [171]:
# if you finish finish, which book is closest to moby dick, but remake your matrix with all vocabulary terms?
# create the term-document matrix
# you'll want to use counters for each book's vocabulary for the sake of efficiency


# you may want to re-define new counter versions of your tf, idf, term_in_documents 
# functions so that they work with counters


In [172]:
# verify the shape of your matrix

In [173]:
# which book is actually closest to moby dick?